Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list anyone you collaborated with on this workbook

---

## Lab 4: Visualization
**This lab was distributed Monday 9/23/2019 and should be completed by Monday 9/30/2019 at 11:59PM.**

Welcome to your fourth lab of the semester!<br>

This lab aims to get you started with visualization in Python.

## Setup

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
%matplotlib inline

## Question 1: Warmup
Let's start by looking at some example figures (your group will be assigned one of these four):<br>
1. [Overnight stays at US National Parks](http://jordan-vincent.com/night-under-the-stars)
1. [Most popular fields of study](https://flowingdata.com/2016/12/07/fields-of-study-ranked-over-past-few-decades/)
1. [Carbon emissions and oceans](https://www.nationalgeographic.com/environment/2019/06/see-the-drastic-toll-climate-change-is-taking-on-our-oceans/)
1. [Tracking Britain's use of coal](https://www.theguardian.com/environment/ng-interactive/2019/may/25/the-power-switch-tracking-britains-record-coal-free-run)

For links that contain more than one visualization, you can choose to comment on just one or multiple visualizations.

### Question 1.1
What does this visualization show? Is there a message that's being conveyed with the visualization? If so, what is it?

*Your answer here*

### Question 1.2
What do you like about this figure? What can be improved?

*Your answer here*

## Question 2: Understanding our data
The data from this lab comes from the Environmental Protection Agency (EPA). From the [daily air quality data tool](https://www.epa.gov/outdoor-air-quality-data/download-daily-data), we pulled PM2.5 data from 2019 for monitors in Alameda County. Let's take a look at the data in our data folder by running the cell below.

In [ ]:
os.listdir('data')

### Question 2.1
Create a dataframe called `alameda_pm25` that contains the data from the .csv found in the 'data' folder.

In [ ]:
alameda_pm25 = ... # YOUR CODE HERE

In [ ]:
alameda_pm25.head()

### Question 2.2
Use `.describe()` to summarize the data in `alameda_pm25`.

In [ ]:
# YOUR CODE HERE

You'll notice that the output of `.describe()` drops the "Date" column because it isn't a numeric type. Try running `.describe()` again, this time setting the argument `include = 'all'`.

In [ ]:
# YOUR CODE HERE

Now, try converting the "Date" field to a `datetime` data type using `pd.to_datetime`. Then, re-run the `.describe` function, using the argument `include = all`. Does the output of `.describe` change when you do this? Why?

In [ ]:
alameda_pm25["Date"] = ... # YOUR CODE HERE
alameda_pm25.describe(include = 'all')

When we assign data type `datetime` to the "Date" field, Python is able to identify additional descriptive statistics like the first and last date - it can't do that when it's just reading a string.

### Question 2.3
What are the unique site names in `alameda_pm25`? 

In [ ]:
# YOUR CODE HERE

## Question 3: Plotting distributions
In this question, we'll create a histogram of PM2.5 concentrations for just the Oakland West site measurements. 

### Question 3.1
Start by creating a dataframe that contains just the columns "Date" and "Daily Mean PM2.5 Concentration" for Oakland West, and name it `oak_west`.

In [ ]:
oak_west = ... # YOUR CODE HERE
oak_west.head()

### Question 3.2
We can use [plt.hist()](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html) to create a histogram of PM2.5 concentrations in Oakland West. First, we'll call the function with just the `x` argument populated. Run the cell below to see what the histogram looks like.

In [ ]:
# run this cell
plt.hist(oak_west["Daily Mean PM2.5 Concentration"])

Let's play around with the `bins` argument `plt.hist()`. As the documentation for `plt.hist()` outlines, `bins` can take a bunch of different datatypes - it can be an integer, or a list of integers, or even a string. Let's try all three of these possibilities below.

In [ ]:
# setting bins to an integer
plt.hist(oak_west["Daily Mean PM2.5 Concentration"], bins = 5)

In [ ]:
# setting bins to a list
bins_list = [i for i in range(0,30,2)]
plt.hist(oak_west["Daily Mean PM2.5 Concentration"], bins = bins_list)

In [ ]:
plt.hist(oak_west["Daily Mean PM2.5 Concentration"], bins = "sqrt")

Try playing around with the values passed to `bins = ` above. Can you explain how each approach bins the data? (the string argument is a little tricky, but it's documented in the ["Notes" section of this page](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram_bin_edges.html))

*Your answer here*

### Question 3.3
Plot a histogram of PM2.5 concentrations for Oakland West below. 
1. Play around with different binning strategies and choose the one you like best.
2. Add axis labels and a title.

In [ ]:
# Fill in code below
plt.hist(...)
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)

### Question 3.4
Why did you choose the binning strategy you did?

*Your answer here*

## Question 4: Creating Kernel Density Estimate (KDE) plots
A Kernel Density Estimate (KDE) gives us a probability distribution for a continuous variable from a set of observed data points. Our histogram gives us some information about the distribution of PM2.5 in West Oakland - we can see how often we observe a PM2.5 concentration that falls within a certain bin over the first 9 months of 2019. But because histograms are binned and discretize data, the amount of information we can get about a continuous variable will always be limited by the size of the bins.<br>

To understand how to create a KDE, we have to first figure out what "kernel" means. A kernel is a weighting function and it shares certain mathematical properties with probability density functions (it always evaluates to a non-negative and real value and its integral - or the area under the curve - equals 1).  Kernels are also *even*, meaning the value they return is independent of the sign of the argument. ([source](https://chemicalstatistician.wordpress.com/2013/06/09/exploratory-data-analysis-kernel-density-estimation-in-r-on-ozone-pollution-data-in-new-york-and-ozonopolis/)).<br>

KDE "applies" the kernel on a grid of points along the support of your data set.  Like the edges of bins in a histogram, these points need not correspond to actual observations in your data set.  They are just equally spaced points, as they were in the histogram examples above.  Now, what does it mean to "apply" the kernel to a point on the support?  Put simply, the kernel measures how many observations in the data set are close by.  The kernel returns a higher number if the point is in a crowded part of the support, and a lower value if the point is out in the wilderness.  Different kernels weight the importance of nearby observations differently.  Some give equal weight to all observations in a range, and others give more weight to observations at the center of the kernel, i.e. the point where you're evaluating the kernel.

What's nice about the construction of KDE kernels is that the collective evaluation of the kernel along all the grid points you chose on the support can be interpreted as a probability distribution.  (This is because the kernels are carefully defined in mathematical terms to ensure this resulting property.)  That is, if you plot all those kernel evaluations as a function of the points where you evaluated them and then connect those points together, the resulting curve is an estimate of the probability density function of the underlying data.  Below is an example of what a KDE might look like for ozone concetrations measured at two locations ([source](https://chemicalstatistician.wordpress.com/2013/06/09/exploratory-data-analysis-kernel-density-estimation-in-r-on-ozone-pollution-data-in-new-york-and-ozonopolis/)).<br>
<img src="images/kernel-density-plot-ozone3.png">

In practice, you would implement a KDE by choosing a kernel and then applying it at regular intervals across your dataset. The length of those intervals simply the width of your support divided by the number of points in your grid -- the *gridsize*, a parameter that you get to select. You would then add up all the probability distribution functions and divide your function by the total number of data points to normalize it - since we're producing a probability distribution, the area under the curve needs to be equal to one.<br>

(If this feels a little confusing, don't worry! We'll discuss it as a group in lab, and the next few questions will also help us understand how KDEs work).

### Question 4.1
Let's use [this tool](https://mathisonian.github.io/kde/) to help us explore KDEs. Play around with the bandwidth and kernel. What happens to the KDE curve (the blue line) when you change the bandwidth? Why? How about the shape of the kernel (the red line)?

*Your answer here*

While it's always good to understand the math behind what we're plotting or analyzing, there are thankfully functions and packages that neatly do the math for us. In the Setup cell at the very top, we imported the package `seaborn` as `sns`. [`seaborn`](https://seaborn.pydata.org/) is a data visualization library that builds on `matplotlib`, and it contains convenient functions for a range of plot types, including KDE plots.

Let's start by running the cell below to pull the documentation for [`kdeplot`](https://seaborn.pydata.org/generated/seaborn.kdeplot.html).

In [ ]:
sns.kdeplot?

### Question 4.2  
From the documentation and from what you know about KDE plots, which argument(s) would you play with to change how the final KDE curve looks?

*Your answer here*

### Question 4.3
Let's draw a KDE plot using `sns.kdeplot()` of PM2.5 concentrations in Oakland West. In the cell below, we're just passing our data to the function, and using default values for all other arguments. Run the cell to see what the plot looks like.

In [ ]:
# run this cell
sns.kdeplot(oak_west["Daily Mean PM2.5 Concentration"])

Try plotting the KDE plot with different gridsize and bandwidth parameters. How does changing the bandwidth parameter impact the plot? Show at least two examples with different bandwidths, and add axis labels and titles for each.

In [ ]:
# Fill in code below
sns.kdeplot(...)
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)

In [ ]:
# Fill in code below
sns.kdeplot(...)
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)

*Your answer here*

### Question 4.4
Grab the data from Livermore by creating a dataframe that contains just the columns "Date" and "Daily Mean PM2.5 Concentration" for Livermore from `alameda_pm25`, and save it to dataframe `livermore`.

Plot the KDE for Livermore on the same plot as West Oakland. Make sure to add a legend labelling the two locations, as well as axis labels and a title.

You can plot multiple curves on the same plot by calling the `sns.kdeplot()` function twice, i.e.:
```python
sns.kdeplot(...) # plot dataset 1
sns.kdeplot(...) # plot dataset 2
```

You can add a legend by passing a list to the plt.legend() command, i.e.:
``` python
plt.legend(["data label 1", "data label 2"])
```

In [ ]:
# Fill in code below
livermore = ...

sns.kdeplot(...)
sns.kdeplot(...)
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)
plt.legend(...)

## Question 5: Box and Whisker Plots
We've tried two different ways to visualize distributions: histograms and probability density functions using Kernel Density Estimates. Another way to show a distribution is a box and whisker plot. It's a much more summarized version of your distribution. The general format of a box and whisker plot is shown below ([source](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_SummarizingData/BS704_SummarizingData8.html)).<br>

<img src = "images/BoxWhisker1.png" width=600>

The "box" shows the bounds of your 25th and 75th percentile (also called the 1st and 3rd quartiles, or the interquartile range (IQR)). Basically, if you ordered your data and then divided it into four bins of equal size, your box would encompass the middle two bins. <br>

The location of the whiskers can vary, but they're intended to show the full range of the data. In Seaborn, the [`sns.boxplot()`](https://seaborn.pydata.org/generated/seaborn.boxplot.html) function has a parameter called "whis". By default, "whis" is equal to 1.5.  This positions the whiskers at 1.5 times the distance from the median to the 25th and 75th percentiles. In this case we say the whiskers span 1.5 times the "interquartile range."  Many scientific papers use this default.  You can also set `whis = [lo hi]` where `lo` and `hi` are numbers between zero and 100 and represent the position of the whiskers by percentile of the distribution. (For example, `whis = [5, 95]` would position the whiskers at the 5th and 95th percentiles).  <br>

We're going to create a box and whisker plot that shows the distribution of PM2.5 values in two locations: Oakland West and Livermore. In order to plot box and whisker plots with multiple categories of data (in this case, multiple site locations), we need each category to have the same length. In our case, `oak_west` contains 2 less observations than `livermore` - you can confirm this using the `.shape` method. To standardize the size, we're going to use `pd.merge()` to combine `oak_west` and `livermore`.

### Question 5.1 
What kind of merge (inner, outer, right, or left) should we do so that we retain all of the data in both `oak_west` and `livermore`? If you're not sure, consult the lab 3 notebook.

*Your answer here*

### Question 5.2 
Merge `oak_west` and `livermore` on the "Date" field, and save the merged dataframe to `pm25_merged`. Rename the columns to something more descriptive (i.e. a column name that includes the site name).

In [ ]:
pm25_merged = ...
# YOUR CODE HERE

In [ ]:
pm25_merged.head()

### Question 5.3
Let's start by running the cell below to plot a basic boxplot using default parameters.

In [ ]:
# run this cell
sns.boxplot(data = pm25_merged.iloc[:,1:])

Let's try using seaborn's syntax for creating and formatting plots. While you can use matplotlib syntax with seaborn, as we did with the KDE plots, seaborn has its own function - [`seaborn.set`](https://seaborn.pydata.org/generated/seaborn.set.html) - that's used for customizing aesthetics. 

In [ ]:
# run this cell
fig = sns.boxplot(data = pm25_merged.iloc[:,1:])

fig.set(
    xlabel = 'Measurement location', 
    ylabel = 'Daily mean PM 2.5 concentration ($\mu g/m^3$)'
)

Now, using the code in the previous cell as a starting point, change the orientation of the plot from vertical to horizontal (hint: check the `sns.boxplot` documentation) and adding a title.

In [ ]:
# Your code here

### Question 5.4
Play around with the boxplot parameters - at minimum, try changing the whiskers "whis" parameter in the boxplots. Plot one of your favorite graphs below, and explain how the changes you made might change how the viewer interprets the data.

In [ ]:
# your code here

*Your answer here*

## Question 6: Line graphs
Let's plot PM2.5 concentrations throughout 2019 to date by creating a line graph with date on the x-axis, and PM2.5 concentration on the y-axis, for all sites. We'll start by just calling the `plt.plot()` function with the x and y data as arguments, using default parameters otherwise.<br>

In [ ]:
# run this cell
plt.plot(alameda_pm25["Date"], alameda_pm25["Daily Mean PM2.5 Concentration"])

This plot is a little confusing! The date labels are crowding each other on the x-axis, and it's hard to identify trends since there are so many lines. We also have back and forth lines because matplotlib is plotting all of the different sites at once, so it's joining data from one site in September 2019 to data from another site in January 2019.<br> 

Let's try assigning colors to the different locations. There are a lot of ways to do this - one method, of looping through all your categories and plotting them in layers, is shown below.<br>

We can start by using a pre-defined color palette from one of [`seaborn`'s color palettes](https://seaborn.pydata.org/tutorial/color_palettes.html) (note that this is the only `seaborn` feature we're using in the line plot code - otherwise, we're just using `matplotlib` syntax). I'm going to choose "colorblind", but you can feel free to change the palette. If we use the `sns.color_palette` method, we can see that "colorblind" is a list of RGB color values. We can assign this color palette to the variable `color`.

In [ ]:
# run this cell
palette = "colorblind" # you can try changing this to another pre-set palette
sns.color_palette(palette)

In [ ]:
# run this cell
color = sns.color_palette(palette)

In the cell below, we're going to loop through the unique site names in the full Alameda county dataset, and plot each dataframe subset separately, assigning a color for each from the list of colors provied in the seaborn color palette.

In [ ]:
# run this cell
sites = list(alameda_pm25["Site Name"].unique())
for name in sites:
    site_df = alameda_pm25[alameda_pm25["Site Name"] == name]
    plt.plot(site_df["Date"], site_df["Daily Mean PM2.5 Concentration"], 
             color = color[sites.index(name)])

That's a bit better - we can see that we've plotted different colors for the different sites now. Let's make the plot bigger and add a legend. To do that, we're going to change our syntax slightly from previous examples. We'll create a new figure by calling `plt.figure()`, to which we can pass an argument for `figsize` that specifies the size of the figure.<br> 

A figure in `matplotlib` can include subplots - by convention, these are often given a variable name `ax`. We only need one plot in our figure, so when we call the function [`fig.addsubplot()`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure.add_subplot), we can just use default arguments (check the function documentation for information on how you can add multiple subplots).<br>

Now, instead of using `plt.plot()`, we're going to use `ax.plot()` since `ax` is the variable that represents our plot within the figure `fig`.<br>

After we've plotted all of the lines, we can add axis labels, a legend, and a title by using the respective methods on `ax`.

In [ ]:
# run this cell
fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot()

for name in sites:
    site_df = alameda_pm25[alameda_pm25["Site Name"] == name]
    ax.plot(site_df["Date"], site_df["Daily Mean PM2.5 Concentration"], color = color[sites.index(name)])

ax.legend(sites)
ax.set_xlabel("Date", fontsize = 12)
ax.set_ylabel("Daily mean PM 2.5 concentration ($\mu g/m^3$)", fontsize = 12)
ax.set_title("Daily mean PM2.5 concentrations in Alameda County, 2019", fontsize = 14)

The plot above is still pretty busy! Try outputting a line plot where the data points for just "Berkeley Aquatic Park" are highlighted in a different color than the rest (eg. Berkeley Aquatic Park in green, everything else in grey - or whatever other color combination makes sense and makes you happy). Use the code in the cell above as a starting point. Make any other tweaks that you think make the plot look better. Do you notice anything about the measurements at Berkeley Aquatic Park compared to the other sites?

In [ ]:
# Your code here

*Your answer here*

# Hooray, you're done! 

Please remember to submit your lab work, after clicking Kernel -> Restart & Run All, in .html and .ipynb format on bCourses.